In [1]:
#Import Dependencies
import pandas as pd
import sqlite3
from sqlite3 import Error


In [2]:
# Read in CSVs
year = [2018, 2019, 2020, 2021]
df_names =[]
df_list = []


for y in year:
    filename = f"{y}weeklystats.csv"
    filepath = f"../DataBase/Resources/weekly_stats/"
    df = f"{y}_weekly_df"
    df_names.append(df)
    read_df = pd.read_csv(f"{filepath}{filename}")
    df_list.append(read_df)

      

In [3]:
#Check df names
df_names

['2018_weekly_df', '2019_weekly_df', '2020_weekly_df', '2021_weekly_df']

In [4]:
#check data list 
len(df_list)

4

In [5]:
#Add year to each dataframe
for i in range(0,len(df_list)):
    yr = df_names[i][:4]
    df_list[i]["Year"] = yr

In [6]:
#Check year was added
for i in range(0,len(df_list)):
    print(df_list[i].info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2500 entries, 0 to 2499
Data columns (total 19 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Rank                  2500 non-null   int64  
 1   Name                  2500 non-null   object 
 2   Team                  2500 non-null   object 
 3   Position              2500 non-null   object 
 4   WeeklyRank            2500 non-null   int64  
 5   Week                  2500 non-null   int64  
 6   Opponent              2500 non-null   object 
 7   PassingYards          2500 non-null   int64  
 8   PassingTouchdowns     2500 non-null   int64  
 9   PassingInterceptions  2500 non-null   int64  
 10  RushingYards          2500 non-null   int64  
 11  RushingTouchdowns     2500 non-null   int64  
 12  Receptions            2500 non-null   int64  
 13  ReceivingYards        2500 non-null   int64  
 14  ReceivingTouchdowns   2500 non-null   int64  
 15  FumblesLost          

In [7]:
#Create empty DF to concatenate position
weekly_stats_df = pd.DataFrame()

In [8]:
#Use coniditionals to concat dataframs
for i in range(0,len(df_list)):
    weekly_stats_df = pd.concat([df_list[i], weekly_stats_df], ignore_index=True)
    print(f"index {i} was added to df")


index 0 was added to df
index 1 was added to df
index 2 was added to df
index 3 was added to df


In [9]:
#Quick Check on QBs
weekly_stats_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 20 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Rank                  10000 non-null  int64  
 1   Name                  10000 non-null  object 
 2   Team                  10000 non-null  object 
 3   Position              10000 non-null  object 
 4   WeeklyRank            10000 non-null  int64  
 5   Week                  10000 non-null  int64  
 6   Opponent              10000 non-null  object 
 7   PassingYards          10000 non-null  int64  
 8   PassingTouchdowns     10000 non-null  int64  
 9   PassingInterceptions  10000 non-null  int64  
 10  RushingYards          10000 non-null  int64  
 11  RushingTouchdowns     10000 non-null  int64  
 12  Receptions            10000 non-null  int64  
 13  ReceivingYards        10000 non-null  int64  
 14  ReceivingTouchdowns   10000 non-null  int64  
 15  FumblesLost         

Start SQL Imports

In [10]:
#Start SQL connections
con = sqlite3.connect("./sqlite_db.sqlite")

In [11]:
weekly_stats_df.to_sql("Weekly_Stats", con, if_exists="replace")

In [12]:
cursor = con.cursor()

In [13]:
# Getting all tables from sqlite_master
sql_query = """SELECT name FROM sqlite_master WHERE type='table';"""

In [14]:
# executing our sql query
cursor.execute(sql_query)
print("List of tables\n")
    
# printing all tables list
print(cursor.fetchall())

List of tables

[('QuickStats',), ('QB_Stats',), ('WR_Stats',), ('RB_Stats',), ('TE_Stats',), ('Teams',), ('Weekly_Stats',)]


In [15]:
cursor.execute("SELECT * FROM Weekly_stats")
rows = cursor.fetchall()
for row in rows:
    print(row)

(0, 22, 'Kyler Murray', 'ARI', 'QB', 1, 1, 'TEN', 289, 4, 1, 20, 1, 0, 0, 0, 0, 33.6, 33.56, '2021', None)
(1, 24, 'Patrick Mahomes', 'KC', 'QB', 2, 1, 'CLE', 337, 3, 0, 18, 1, 0, 0, 0, 0, 33.3, 33.28, '2021', None)
(2, 47, 'Jameis Winston', 'NO', 'QB', 3, 1, 'GB', 148, 5, 0, 37, 0, 0, 0, 0, 0, 29.6, 29.62, '2021', None)
(3, 55, 'Jared Goff', 'DET', 'QB', 4, 1, 'SF', 338, 3, 1, 14, 0, 0, 0, 0, 0, 28.9, 28.92, '2021', None)
(4, 57, 'Jalen Hurts', 'PHI', 'QB', 5, 1, 'ATL', 264, 3, 0, 62, 0, 0, 0, 0, 0, 28.8, 28.76, '2021', None)
(5, 76, 'Dak Prescott', 'DAL', 'QB', 6, 1, 'TB', 403, 3, 1, 13, 0, 0, 0, 0, 0, 27.4, 27.42, '2021', None)
(6, 82, 'Tom Brady', 'TB', 'QB', 7, 1, 'DAL', 379, 4, 2, 0, 0, 0, 0, 0, 0, 27.2, 27.16, '2021', None)
(7, 85, 'Russell Wilson', 'SEA', 'QB', 8, 1, 'IND', 254, 4, 0, 9, 0, 0, 0, 0, 0, 27.1, 27.06, '2021', None)
(8, 105, 'Tyreek Hill', 'KC', 'WR', 9, 1, 'CLE', 0, 0, 0, 4, 0, 11, 197, 1, 0, 26.1, 26.1, '2021', None)
(9, 108, 'Amari Cooper', 'DAL', 'WR', 10, 1, '